# DESCRIPTION
This notebook reads in the data and runs the "month averaging" linear regression for each grid cell. The notebook is designed to be executed with 10 processors as it decomposes the 2D domain in the x-dimension

In [ ]:
"""
    IMPORT LIBRARIES
"""
import xarray as xr
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import os
import pyproj
import rioxarray as rxr
import datetime
import pandas as pd
import tensorflow as tf

In [2]:
"""
    INPUTS
"""
## processor identification ##
me = 0
## total number of processors ##
num_proc = 10 
## output filepath ##
fpath_out = '/glade/scratch/rossamower/snow/snowmodel/aso/california/tuolumne/snowmodel/cheyenne/outputs/aso_sm/ml_results/grid/take_2/'



In [3]:
"""
    LOAD DATASET
"""
ds_output_dir = '/glade/scratch/rossamower/snow/snowmodel/aso/california/tuolumne/snowmodel/cheyenne/outputs/aso_sm/'
ds = xr.load_dataset(ds_output_dir + 'aso_sm_merge_clean_2.nc')

In [4]:
def preprocess(df,testyr,trainyr,isOneHot = True):
    """
        FUNCTION TAKES IN DATA AND CREATES TEST AND TRAINING DATASET
    """
    totyrs = trainyr + testyr
    if isOneHot == True:
        df_totyrs = df[df.date_year.isin(totyrs)]
        if 2015 in testyr:
            df3 = df_totyrs[df_totyrs.date_month != 2]
            df3 = df3[df3.date_month != 7]
        elif 2017 in testyr:
            df3 = df_totyrs[df_totyrs.date_month != 2]
            df3 = df3[df3.date_month != 1]
            df3 = df3[df3.date_month != 8]
            df3 = df3[df3.date_month != 4]
        else:
            df3 = df_totyrs[df_totyrs.date_month != 2]
            df3 = df3[df3.date_month != 1]
            df3 = df3[df3.date_month != 8]
        df_hot1 = pd.get_dummies(data=df3, columns=['date_month'],drop_first = False)
    else:
        df_hot1 = df
        
    ## train/test ##
    df_train = df_hot1[df_hot1.date_year.isin(trainyr)]
    df_test = df_hot1[df_hot1.date_year.isin(testyr)]
    ## drop na's ##
    df_tn_nona = df_train.dropna(axis = 0, how = 'any')
    df_ts_nona = df_test.dropna(axis = 0, how = 'any')
    ## create index vector ##
    index_ = df_ts_nona.index
    ## pull out labels ##
    y_train_ = df_tn_nona.aso_swe
    y_test_ = df_ts_nona.aso_swe
    ## drop columns ##
    if isOneHot == True:
        df_tn_nona_ = df_tn_nona.drop(columns = ['aso_swe','date_year'])
        df_ts_nona_ = df_ts_nona.drop(columns = ['aso_swe','date_year'])
    else:
        df_tn_nona_ = df_tn_nona.drop(columns = ['aso_swe','date_year','date_month'])
        df_ts_nona_ = df_ts_nona.drop(columns = ['aso_swe','date_year','date_month'])
    
    return df_tn_nona_,y_train_,df_ts_nona_,y_test_,index_,df_test

In [5]:
def reg_stats(y,yhat,X):
    """
        FUNCTION CALCULATES R2 AND ADJUSTED R2 VALUES
    """
    SS_Residual = sum((y-yhat)**2)       
    SS_Total = sum((y-np.mean(y))**2) 
    if SS_Total == 0.0:
        r_squared = 0.0
    else:
        r_squared = 1 - (float(SS_Residual))/SS_Total
    if len(y)-X.shape[1]-1 == 0:
        adjusted_r_squared = -1.0
    else:
        adjusted_r_squared = 1 - (1-r_squared)*(len(y)-1)/(len(y)-X.shape[1]-1)
    
    return r_squared,adjusted_r_squared

In [6]:
from sklearn.linear_model import LinearRegression

def lr1(X,y):
    """
        FIT LINEAR REGRESSION USING SKLEARN
    """
    model = LinearRegression()
    model.fit(X, y)
    
    return model

In [7]:
def log_inverse(arry):
    """
        INVERSE LN + 1 TRANSFORMATION
    """
    arry =  np.exp(arry) - 1
    return arry

In [8]:
def grid_lr(df,testyr,trainyr,isOneHot):
    """
        FUNCTION GETS THE TRAINING AND TESTING DATA AND THEN PERFORMS LINEAR REGRESSION AND MODEL STATISTICS
    """
    ## call function to get cleaned training and testing dataframes ##
    x_train, y_train, x_test, y_test, index,df_test1 = preprocess(df,testyr,trainyr,isOneHot)
    ## fit model ##
    model = lr1(x_train,y_train)
    ## predictions ##
    baseline_yhat_tf = model.predict(x_test)
    ## get stats ##
    r_squared,adjusted_r_squared = reg_stats(y_test,baseline_yhat_tf,x_test)
    ## model parameters ##
    bias = model.intercept_
    coef = model.coef_
    yhat_bl = baseline_yhat_tf.flatten()
    ## log inverse ##
    yhat_bl = log_inverse(yhat_bl)
    
    return df_test1,index,yhat_bl,bias,coef,r_squared

In [9]:
def data_merge(pred,name,index,df_orig,lst):
    """
        MERGE DATAFRAMES FROM THE CROSS-VALIDATION SAMPLE
    """
    result = pd.DataFrame(data = {name:pred},
                              index = index)
    
    df_1 = pd.merge(df_orig, result, 
                    left_index = True, right_index = True, 
                    how="left", indicator=False)
    
    lst.append(df_1)
    return lst

In [10]:
def create_dataframe(X,y,date):
    """
        FUNCTION CREATES DATAFRAME FOR GRID CELL
    """
    ## initiate dataframe ##
    df = pd.DataFrame({'date_t':date,
            'sm_swe':X,
            'aso_swe':y}) 
    ## create month/year variable ##
    df['month_year'] = df['date_t'].dt.to_period('M')
    ## groupby month/year variable ##
    df_gp = pd.DataFrame(df.groupby('month_year')[['aso_swe','sm_swe']].mean())
    ## reset index ##
    df_gp = df_gp.reset_index()
    ## create year and month column ##
    df_gp['date_year'] = df_gp['month_year'].dt.year
    df_gp['date_month'] = df_gp['month_year'].dt.month
    ## drop month/year ##
    df_gp = df_gp.drop(columns = ['month_year'])
    
    return df_gp
    

In [11]:
def run_grid(ds,i,j,y_val):
        """
            LOOPS OVER CROSS-VALIDATION TRAIN/TESTING SAMPLES AND PERFORMS LINEAR REGRESSION AT EACH GRID CELL
        """
        ## pull out X and datetime values ##
        x_val = np.log1p(ds.sm_swe[:,j,i].values,dtype=np.float128)
        date_t = np.array(pd.to_datetime(ds.time[:].values))
        date_month = np.array(pd.to_datetime(ds.time[:].values).month)
        date_year = np.array(pd.to_datetime(ds.time[:].values).year)     
        ## create initial dataframe with X,y and timing ##
        df1 = create_dataframe(x_val,y_val,date_t)
        ## break down of train and test years ##
        yr_list = [[2015,2016],[2017,2018]]
        yr_test = [2019,2020]
        training_yrs = [2013,2014]
        ## create lr statistic lists ##
        bias_bl_lst = []
        r2_bl_lst = []
        df_lst_bl = []
        ## start cross-validation loop ##
        for yr in yr_list:
            testing_yrs = yr
            ## create dataframe for grid ##
            isOneHot = False
            df_test1,index,yhat_bl,bias_bl_tn,coef_bl_tn,r2_bl_tn = grid_lr(df1,testing_yrs,
                                             training_yrs,isOneHot)
            ## merge results to dataframe ##
            df_lst_bl = data_merge(yhat_bl,'yhat_bl',index,df_test1,df_lst_bl)
            ## append training variables ##
            training_yrs.append(yr[0])
            training_yrs.append(yr[1])
            bias_bl_lst.append(bias_bl_tn)
            r2_bl_lst.append(r2_bl_tn)
        ## run last validation ##  
        isOneHot = False
        df_test1,index,yhat_bl,bias_bl_ts,coef_bl_ts,r2_bl_ts = grid_lr(df1,yr_test,
                                         training_yrs,isOneHot)
        ## merge results to dataframe ##
        df_lst_bl = data_merge(yhat_bl,'yhat_bl',index,df_test1,df_lst_bl)
        
        ## append and clean results ##
        bias_bl_lst.append(bias_bl_ts)
        r2_bl_lst.append(r2_bl_ts)
        df_2_bl = pd.concat(df_lst_bl)
        df_3_bl = df_2_bl.fillna(-9999.0)
        return bias_bl_lst,r2_bl_lst,list(coef_bl_ts),df_3_bl

In [12]:
def lst_to_output(lst,name,fpath,me,toFile=True):
    """
        FUNCTION CONVERTS LISTS TO NUMPY ARRAYS, PERFORMS CLEANING, AND OUTPUTS TO .GDAT FILES
    """
    arr = np.array(lst,dtype = np.float64)
    arr[arr == -9999.0] = np.nan
    if toFile == True:
        arr.tofile(fpath + name +'_ln_'  + str(me) +'_.gdat')
    return arr

In [13]:
"""
    RUN MODEL
"""
from IPython.display import clear_output
import time
start = time.time()
import sys 

## 1D-decomp identification ##

if me == 0:
    i_start = 0
    i_end = i_start + 101
elif me == num_proc:
    i_start = (me * 100) + 1
    i_end = i_start + 45
else:
    i_start = (me * 100) + 1
    i_end = i_start + 100
    
## instantiate lists ##
bl_lst = []
sm_lst = []
aso_lst = []
bl_r2_lst = []
bl_bias_lst = []
bl_coef_lst = []


## create pd datatime that is grouped by year and month to join data to ##
date_pd = pd.DataFrame({'time':pd.to_datetime(ds.time[:].values)})#,
date_pd['month_year'] = date_pd['time'].dt.to_period('M')
date_pd['blank'] = 1
df_new = pd.DataFrame(date_pd.groupby('month_year')['blank'].mean())
df_new = df_new.reset_index()
## start iteration ##
for i in range(i_start,i_end):
    print(i,end = ' ')
    for j in range(ds.y.shape[0]):
        ## condition for grid cells where aso data is NaN ##
        if np.isnan(ds.notGrid[j,i].values) == True: # dont run model 
            yhat_bl = list(np.full((23), -9999.0))
            sm_bl = list(np.full((23), -9999.0))
            aso_bl = list(np.full((23), -9999.0))
            r2_bl = list(np.full((3), -9999.0))
            coef_bl = list(np.full((1), -9999.0))
            bias_bl = list(np.full((3), -9999.0))
            
        else:
            y_val = np.log1p(ds.aso_swe[:,j,i].values,dtype=np.float128)
            try:
                ## grid regression ##
                bias_bl,r2_bl,coef_bl,df_bl = run_grid(ds,i,j,y_val)
                ## grid merge and post-process ##
                df_bl_j = df_new.join(df_bl,how = 'outer')
                df_bl_j = df_bl_j[df_bl_j['sm_swe'].notna()]
                df_bl_j = df_bl_j.fillna(-9999.0)
                yhat_bl = df_bl_j['yhat_bl'].to_list() #yhat_oh
                sm_bl = df_bl_j['sm_swe'].to_list() #yhat_oh
                aso_bl = df_bl_j['aso_swe'].to_list() #yhat_oh


            except:
                yhat_bl = list(np.full((23), -9999.0))
                r2_bl = list(np.full((3), -9999.0))
                coef_bl = list(np.full((1), -9999.0))
                bias_bl = list(np.full((3), -9999.0))
                sm_bl = list(np.full((23), -9999.0))
                aso_bl = list(np.full((23), -9999.0))

        for val in range(0,len(yhat_bl)):
            bl_lst.append(yhat_bl[val])
            sm_lst.append(sm_bl[val])
            aso_lst.append(aso_bl[val])
            
        for val in range(0,len(bias_bl)):
            bl_bias_lst.append(bias_bl[val])
            bl_r2_lst.append(r2_bl[val])
            
        for val in range(0,len(coef_bl)):
            bl_coef_lst.append(coef_bl[val])

                
end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

                
## data output ##      
bl_arr = lst_to_output(bl_lst,'bl_yhat',fpath_out,me,toFile=False) 
sm_arr = lst_to_output(sm_lst,'bl_sm',fpath_out,me,toFile=False) 
aso_arr = lst_to_output(aso_lst,'bl_aso',fpath_out,me,toFile=False) 
bl_r2_arr = lst_to_output(bl_r2_lst,'bl_r2',fpath_out,me,toFile=False)  
bl_bias_arr = lst_to_output(bl_bias_lst,'bl_bias',fpath_out,me,toFile=False)  
bl_coef_arr = lst_to_output(bl_coef_lst,'bl_coef',fpath_out,me,toFile=False)
 


print('END LINEAR REGRESSION ---------------->')

0 00:00:00.03
END LINEAR REGRESSION ---------------->
